# 03. Metadata Layer Deep Dive

Iceberg 테이블의 **Metadata Layer**를 구성하는 metadata.json, Manifest List, Manifest File을 직접 읽어보며 그 내부 구조를 분석합니다.

## Metadata Layer 구조

Iceberg의 Metadata Layer는 **3단계 계층 구조**로 이루어져 있습니다.

```
metadata.json (Table Metadata)
  │
  │  current-snapshot-id 로 현재 스냅샷을 가리킴
  │
  ├─▶ snap-{id}.avro (Manifest List)
  │     │
  │     │  각 manifest file의 경로와 요약 통계
  │     │
  │     ├─▶ {hash}.avro (Manifest File)
  │     │     └─ data file 경로, record_count, column_sizes,
  │     │        value_counts, null_value_counts,
  │     │        lower_bounds, upper_bounds
  │     │
  │     └─▶ {hash}.avro (Manifest File)
  │           └─ ...
  │
  └─▶ (이전 스냅샷의 manifest list)
```

### 각 파일의 역할

| 파일 | 포맷 | 내용 |
|------|------|------|
| **metadata.json** | JSON | 테이블의 전체 상태: 스키마, 파티션 스펙, 스냅샷 목록, 현재 스냅샷 ID |
| **snap-*.avro** | Avro | Manifest List: 이 스냅샷에 포함되는 manifest file 목록 + 파티션 요약 통계 |
| **\*.avro** | Avro | Manifest File: 데이터 파일별 상세 정보 (경로, 레코드 수, 컬럼 통계 등) |

## 환경 설정

In [ ]:
import sys
sys.path.append('..')

from utils.spark_setup import create_spark_session
from utils.data_generator import generate_orders, to_spark_df
from utils.file_explorer import show_tree

In [ ]:
spark = create_spark_session("MetadataDeepDive")

## 테이블 생성 및 여러 스냅샷 만들기

INSERT → UPDATE → INSERT 순으로 작업하여 **3개의 스냅샷**을 생성합니다.

In [ ]:
spark.sql("CREATE DATABASE IF NOT EXISTS demo.lab")
spark.sql("DROP TABLE IF EXISTS demo.lab.meta_orders")

spark.sql("""
    CREATE TABLE demo.lab.meta_orders (
        order_id     BIGINT,
        customer_id  BIGINT,
        product_name STRING,
        order_date   DATE,
        amount       DOUBLE,
        status       STRING
    )
    USING iceberg
    PARTITIONED BY (months(order_date))
""")

print("테이블 생성 완료")

In [ ]:
# Snapshot 1: 100건 삽입
orders1 = generate_orders(num_records=100, seed=42)
df1 = to_spark_df(spark, orders1)
df1.writeTo("demo.lab.meta_orders").append()
print("Snapshot 1: 100건 INSERT 완료")

In [ ]:
# Snapshot 2: 10건 UPDATE
spark.sql("""
    UPDATE demo.lab.meta_orders
    SET status = 'cancelled', amount = 0.0
    WHERE order_id <= 10
""")
print("Snapshot 2: 10건 UPDATE 완료")

In [ ]:
# Snapshot 3: 50건 추가 삽입
orders2 = generate_orders(num_records=50, id_offset=101, seed=99)
df2 = to_spark_df(spark, orders2)
df2.writeTo("demo.lab.meta_orders").append()
print("Snapshot 3: 50건 INSERT 완료")

In [ ]:
# 현재 상태 확인
count = spark.sql("SELECT COUNT(*) AS cnt FROM demo.lab.meta_orders").collect()[0]["cnt"]
print(f"현재 총 레코드: {count}건")

spark.sql("SELECT * FROM demo.lab.meta_orders.snapshots").show(truncate=False)

## metadata/ 디렉토리 구조 확인

In [ ]:
TABLE_PATH = "/home/jovyan/data/warehouse/lab/meta_orders"
METADATA_PATH = f"{TABLE_PATH}/metadata"

show_tree(METADATA_PATH)

## 1단계: metadata.json 분석

`metadata.json`은 테이블의 **전체 상태**를 담고 있는 핵심 파일입니다.  
가장 최신 버전의 metadata.json을 직접 읽어봅니다.

In [ ]:
import json
import glob
import os

# 가장 최신 metadata.json 찾기
metadata_files = sorted(glob.glob(f"{METADATA_PATH}/v*.metadata.json"))
latest_metadata = metadata_files[-1]
print(f"최신 metadata 파일: {os.path.basename(latest_metadata)}")
print(f"전체 metadata 파일 수: {len(metadata_files)}개 (= 스냅샷 수 + 테이블 생성)")

with open(latest_metadata) as f:
    meta = json.load(f)

In [ ]:
# 주요 최상위 필드
print("[ metadata.json 최상위 필드 ]\n")
print(f"  format-version:      {meta['format-version']}")
print(f"  table-uuid:          {meta['table-uuid']}")
print(f"  location:            {meta['location']}")
print(f"  current-schema-id:   {meta.get('current-schema-id', 'N/A')}")
print(f"  current-snapshot-id: {meta.get('current-snapshot-id', 'N/A')}")
print(f"  스키마 버전 수:      {len(meta.get('schemas', []))}")
print(f"  스냅샷 수:           {len(meta.get('snapshots', []))}")
print(f"  파티션 스펙 수:      {len(meta.get('partition-specs', []))}")

In [ ]:
# 스키마 정보
print("[ schemas ]\n")
for schema in meta.get('schemas', []):
    print(f"  Schema ID: {schema['schema-id']}")
    for field in schema['fields']:
        print(f"    - id={field['id']}, name={field['name']}, type={field['type']}, required={field['required']}")
    print()

In [ ]:
# 파티션 스펙
print("[ partition-specs ]\n")
for spec in meta.get('partition-specs', []):
    print(f"  Spec ID: {spec['spec-id']}")
    for field in spec['fields']:
        print(f"    - source-id={field['source-id']}, name={field['name']}, transform={field['transform']}")

In [ ]:
# 스냅샷 목록
print("[ snapshots ]\n")
for snap in meta.get('snapshots', []):
    print(f"  Snapshot ID:   {snap['snapshot-id']}")
    print(f"  Timestamp:     {snap['timestamp-ms']}")
    print(f"  Operation:     {snap.get('summary', {}).get('operation', 'N/A')}")
    print(f"  Manifest List: {os.path.basename(snap['manifest-list'])}")
    summary = snap.get('summary', {})
    print(f"  Summary:       added-files={summary.get('added-data-files-count', '?')}, "
          f"total-records={summary.get('total-records-count', '?')}")
    print()

## 2단계: Manifest List (snap-*.avro) 분석

Manifest List는 스냅샷에 포함되는 **manifest file들의 목록**을 담고 있습니다.  
`fastavro`를 사용하여 직접 읽어봅니다.

In [ ]:
import fastavro

# 현재 스냅샷의 manifest list 찾기
current_snap = None
current_snap_id = meta.get('current-snapshot-id')
for snap in meta.get('snapshots', []):
    if snap['snapshot-id'] == current_snap_id:
        current_snap = snap
        break

manifest_list_path = current_snap['manifest-list']
print(f"현재 스냅샷의 Manifest List: {os.path.basename(manifest_list_path)}\n")

with open(manifest_list_path, 'rb') as f:
    reader = fastavro.reader(f)
    manifest_list_records = list(reader)

print(f"Manifest 파일 수: {len(manifest_list_records)}개\n")

for i, record in enumerate(manifest_list_records):
    print(f"  [{i}] Manifest File")
    print(f"      path:                  {os.path.basename(record['manifest_path'])}")
    print(f"      manifest_length:       {record.get('manifest_length', 'N/A')} bytes")
    print(f"      added_data_files_count:   {record.get('added_data_files_count', 'N/A')}")
    print(f"      existing_data_files_count: {record.get('existing_data_files_count', 'N/A')}")
    print(f"      deleted_data_files_count:  {record.get('deleted_data_files_count', 'N/A')}")
    
    partitions = record.get('partitions', [])
    if partitions:
        print(f"      partition_summaries:")
        for ps in partitions:
            print(f"        contains_null={ps.get('contains_null')}, "
                  f"lower_bound={ps.get('lower_bound')}, upper_bound={ps.get('upper_bound')}")
    print()

## 3단계: Manifest File (*.avro) 분석

Manifest File은 개별 **데이터 파일의 상세 정보**를 담고 있습니다.  
여기에 **컬럼 수준 통계**(min/max, null count 등)가 저장됩니다.

In [ ]:
# 첫 번째 manifest file 읽기
manifest_path = manifest_list_records[0]['manifest_path']
print(f"Manifest File: {os.path.basename(manifest_path)}\n")

with open(manifest_path, 'rb') as f:
    reader = fastavro.reader(f)
    manifest_records = list(reader)

print(f"데이터 파일 엔트리 수: {len(manifest_records)}개\n")

# 첫 번째 엔트리 상세 분석
entry = manifest_records[0]
df_info = entry.get('data_file', entry)  # 구조에 따라 다를 수 있음

print("[ 데이터 파일 엔트리 키 목록 ]")
if isinstance(df_info, dict):
    for key in sorted(df_info.keys()):
        val = df_info[key]
        if isinstance(val, dict) and len(str(val)) > 100:
            print(f"  {key}: (dict, {len(val)} entries)")
        elif isinstance(val, bytes) and len(val) > 50:
            print(f"  {key}: (bytes, {len(val)} bytes)")
        else:
            print(f"  {key}: {val}")

In [ ]:
# 모든 데이터 파일 엔트리의 주요 정보 출력
print("[ 모든 데이터 파일 상세 ]\n")

for i, entry in enumerate(manifest_records[:10]):  # 최대 10개만
    df_info = entry.get('data_file', entry)
    if isinstance(df_info, dict):
        file_path = df_info.get('file_path', 'N/A')
        record_count = df_info.get('record_count', 'N/A')
        file_size = df_info.get('file_size_in_bytes', 'N/A')
        
        print(f"  [{i}] {os.path.basename(str(file_path))}")
        print(f"      record_count:     {record_count}")
        print(f"      file_size:        {file_size} bytes")
        
        # column_sizes
        col_sizes = df_info.get('column_sizes', {})
        if col_sizes:
            print(f"      column_sizes:     {col_sizes}")
        
        # value_counts
        val_counts = df_info.get('value_counts', {})
        if val_counts:
            print(f"      value_counts:     {val_counts}")
        
        # null_value_counts
        null_counts = df_info.get('null_value_counts', {})
        if null_counts:
            print(f"      null_value_counts: {null_counts}")
        
        # lower_bounds / upper_bounds
        lower = df_info.get('lower_bounds', {})
        upper = df_info.get('upper_bounds', {})
        if lower:
            print(f"      lower_bounds:     {lower}")
        if upper:
            print(f"      upper_bounds:     {upper}")
        print()

## Column-level Statistics와 쿼리 최적화

Manifest File에 저장된 `lower_bounds`와 `upper_bounds`는 **파일 프루닝(File Pruning)**에 사용됩니다.

예를 들어, 다음 쿼리를 실행할 때:

```sql
SELECT * FROM orders WHERE amount > 5000
```

Iceberg는 각 데이터 파일의 `amount` 컬럼 `upper_bounds`를 확인하여:
- `upper_bounds(amount) < 5000`인 파일은 **읽지 않고 건너뜁니다**
- 이를 통해 불필요한 I/O를 줄이고 쿼리 성능을 향상시킵니다

```
파일 A: amount lower=100, upper=3000  → ❌ SKIP (max < 5000)
파일 B: amount lower=200, upper=6000  → ✅ READ (max >= 5000)
파일 C: amount lower=5500, upper=8000 → ✅ READ (max >= 5000)
```

## Snapshot 체인 시각화

여러 작업을 거치면서 스냅샷이 어떻게 쌓이는지 확인합니다.

In [ ]:
from datetime import datetime

snapshots = meta.get('snapshots', [])

print("Snapshot 체인:")
print("=" * 70)

for i, snap in enumerate(snapshots):
    snap_id = snap['snapshot-id']
    ts = datetime.fromtimestamp(snap['timestamp-ms'] / 1000).strftime('%Y-%m-%d %H:%M:%S')
    op = snap.get('summary', {}).get('operation', '?')
    total_records = snap.get('summary', {}).get('total-records-count', '?')
    is_current = " ◀ CURRENT" if snap_id == current_snap_id else ""
    
    if i > 0:
        print("     │")
        print("     ▼")
    
    print(f"  ┌─── Snapshot v{i+1}{is_current}")
    print(f"  │ ID:        {snap_id}")
    print(f"  │ Time:      {ts}")
    print(f"  │ Operation: {op}")
    print(f"  │ Records:   {total_records}")
    print(f"  └───")

## 관찰 포인트

### Metadata Layer의 핵심 역할

1. **metadata.json**: 테이블의 "전체 역사"를 담는 파일
   - 모든 스냅샷 목록과 현재 스냅샷 ID를 관리
   - 스키마 변경 이력, 파티션 스펙 등을 추적

2. **Manifest List (snap-*.avro)**: 스냅샷의 "목차"
   - 어떤 manifest file들이 이 스냅샷에 포함되는지
   - 파티션별 요약 통계로 빠른 필터링 가능

3. **Manifest File (*.avro)**: 데이터 파일의 "상세 카탈로그"
   - 파일별 레코드 수, 크기, 컬럼 통계
   - `lower_bounds`/`upper_bounds`로 파일 프루닝 지원
   - `null_value_counts`로 NULL 관련 쿼리 최적화

### Metadata가 쿼리에 미치는 영향

- Manifest List의 파티션 요약 → **파티션 프루닝**
- Manifest File의 min/max 통계 → **파일 프루닝**
- 이 두 단계를 거쳐 **실제로 읽어야 할 데이터 파일 수를 최소화**합니다

### 스냅샷 체인

- 각 작업(INSERT, UPDATE, DELETE)마다 새로운 스냅샷이 생성됩니다
- 이전 스냅샷은 그대로 유지되어 **Time Travel**이 가능합니다
- 이 스냅샷 기반 버전 관리를 다음 `3_features/01-time-travel.ipynb`에서 활용합니다

In [ ]:
spark.stop()
print("Spark 세션 종료")